<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Pratical-Keras-Tutorials/How_to_Use_Keras_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building the Model for One Input & Output

## Loading dataset

In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

## Data Preprocessing

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)

In [3]:
import tensorflow as tf
tf.random.set_seed(42)

## Modeling

### Creating the layers

In [4]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

### Building the model

In [5]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)
model = tf.keras.Model(inputs=[input_], outputs=[output])

In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 8)         │         17 │ input_layer[0][0] │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 30)        │        270 │ normalization[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 30)        │        930 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 38)        │          0 │ normalization[0]… │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         39 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,256 (4.91 KB)

 Trainable params: 1,239 (4.84 KB)

 Non-trainable params: 17 (72.00 B)

## Model Training

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - RootMeanSquaredError: 1.2830 - loss: 1.7175 - val_RootMeanSquaredError: 0.7533 - val_loss: 0.5675
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6856 - loss: 0.4702 - val_RootMeanSquaredError: 0.6689 - val_loss: 0.4475
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6255 - loss: 0.3914 - val_RootMeanSquaredError: 0.6461 - val_loss: 0.4174
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.6070 - loss: 0.3686 - val_RootMeanSquaredError: 0.6348 - val_loss: 0.4030
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5966 - loss: 0.3560 - val_RootMeanSquaredError: 0.6248 - val_loss: 0.3904
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5889 - loss: 0.3469 - val_RootMeanSquaredError: 0.6173 - val_loss: 0.3810
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5822 - los

## Model Evaluation

In [8]:
mse_test = model.evaluate(X_test, y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - RootMeanSquaredError: 0.5529 - loss: 0.3059


## Prediction

In [9]:
import pandas as pd
X_new = X_test[:3]
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44


In [10]:
y_pred = model.predict(X_new)
pd.DataFrame(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step


,0
0,0.571223
1,1.267854
2,4.528666


In [11]:
pd.DataFrame(y_test[:3])

,0
0,0.47700
1,0.45800
2,5.00001


# Building the Model with Two Inputs & One Output

## Creating the layers

In [12]:
# Input layers
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

# Normalization layers
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

## Modeling

In [13]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Hidden layers
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenation layer
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Output layer
output = tf.keras.layers.Dense(1)(concat)

# Model building
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

## Model Compiling

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.4578 - loss: 2.2105 - val_RootMeanSquaredError: 0.7831 - val_loss: 0.6132
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.7251 - loss: 0.5261 - val_RootMeanSquaredError: 0.6871 - val_loss: 0.4722
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6516 - loss: 0.4247 - val_RootMeanSquaredError: 0.6573 - val_loss: 0.4320
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6253 - loss: 0.3911 - val_RootMeanSquaredError: 0.6420 - val_loss: 0.4121
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6126 - loss: 0.3754 - val_RootMeanSquaredError: 0.6316 - val_loss: 0.3989
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6026 - loss: 0.3632 - val_RootMeanSquaredError: 0.6235 - val_loss: 0.3887
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5953 - los

## Model Evaluation

In [15]:
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.5743 - loss: 0.3300


In [16]:
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
y_pred = model.predict((X_new_wide, X_new_deep))
pd.DataFrame(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step


,0
0,0.484072
1,1.302008
2,3.898753


# Building the Model with Two Inputs & Two Outputs

### Model Building

In [17]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Hidden layers
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenation layer
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Output layers
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)

model = tf.keras.Model(inputs=[input_wide, input_deep],outputs=[output, aux_output])

## Model Compiling

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1),
              optimizer=optimizer,
              metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.RootMeanSquaredError()])

### Model Training

In [21]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), (y_train, y_train), epochs=20, validation_split=0.2)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - dense_8_loss: 2.9123 - dense_8_root_mean_squared_error: 1.6562 - dense_9_loss: 2.8476 - dense_9_root_mean_squared_error: 1.6676 - loss: 2.9058 - val_dense_8_loss: 0.6256 - val_dense_8_root_mean_squared_error: 0.7908 - val_dense_9_loss: 0.9075 - val_dense_9_root_mean_squared_error: 0.9528 - val_loss: 0.6536
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_8_loss: 0.5362 - dense_8_root_mean_squared_error: 0.7321 - dense_9_loss: 0.7579 - dense_9_root_mean_squared_error: 0.8703 - loss: 0.5584 - val_dense_8_loss: 0.4817 - val_dense_8_root_mean_squared_error: 0.6939 - val_dense_9_loss: 0.6797 - val_dense_9_root_mean_squared_error: 0.8244 - val_loss: 0.5014
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_8_loss: 0.4322 - dense_8_root_mean_squared_error: 0.6573 - dense_9_loss: 0.6063 - dense_9_root_mean_squared_error: 0.7786 - loss: 0.4496 - val_dense_8_loss: 0.4369 - val_dense_8_root_mean_squared_error: 0.6609 - val_

### Model Evaluation

In [22]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - dense_8_loss: 0.3279 - dense_8_root_mean_squared_error: 0.5725 - dense_9_loss: 0.3863 - dense_9_root_mean_squared_error: 0.6215 - loss: 0.3338


### Model Prediction

In [23]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step


In [24]:
y_pred_main

array([[0.45443738],
       [1.204264  ],
       [3.8949058 ]], dtype=float32)

In [25]:
y_pred_aux

array([[0.6158254],
       [1.0057695],
       [3.729509 ]], dtype=float32)

In [26]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [27]:
y_pred

{'dense_8': array([[0.45443738],
        [1.204264  ],
        [3.8949058 ]], dtype=float32),
 'dense_9': array([[0.6158254],
        [1.0057695],
        [3.729509 ]], dtype=float32)}

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) |[GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎

# Resource

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)
